# <center> **Notebook get_fighters()**</center>

Notebook em jupyter que descreve a função `get_fighters()` do scrip ufc_scap() desde o webscraping até os tratamentos realizados para gerar o data frame final.

### Bibliotecas utilizadas

In [1]:
# importando bibliotecas utilizadas no projeto
# Webcraping
import requests 
from bs4 import BeautifulSoup

# Manipulação de dados
import pandas as pd
import numpy as np

# Avisos ao usuario
import tkinter as tk
from tkinter import messagebox

## Webscraping

O webscraping executado nesta função requer modificações quando comparado ao realizado na função get_events(), principalmente devido à configuração do site que categoriza os atletas em ordem alfabética, desde "a" até "z".

In [2]:
# Criando função que retorna os lutadores de uma determinada letra

def letter_get_fighters(letra):
    """
    Função que realiza o scrap da pagina e retorna um dataframe pandas com a letra especificada

    Obs: A letra deve esta minuscula
    """

    # Link de acesso para pagina com a letra especificada
    URL = f"http://www.ufcstats.com/statistics/fighters?char={letra}&page=all"

    # Realizando request para o site
    response = requests.get(URL)
    response.raise_for_status()

    # Obtendo pagina completa em html
    soup = BeautifulSoup(response.content, "html.parser")

    # Encontrando tabela com a classe especificada
    table = soup.find("table", class_="b-statistics__table")

    # Transformando tabela de html para dataframe pandas ou retornando erro se não encontrar a tabela
    if table:
        # Extraindo cabeçalho
        header = [th.text.strip() for th in table.find('thead').find_all('th')]

        # Extraindo as linhas
        rows = table.find('tbody').find_all('tr')
        table_data = [[col.text.strip() for col in row.find_all('td')] for row in rows]

        # Convertendo em dataframe
        df_bruto = pd.DataFrame(table_data[1:], columns=header) #Obs: Não considera a primeira linha

        return df_bruto
    else:
        return print("Não foi possível encontrar a tabela desejada na página.")

In [3]:
letter_get_fighters("a")

,First,Last,Nickname,Ht.,Wt.,Reach,Stance,W,L,D,Belt
0,Tom,Aaron,,--,155 lbs.,--,,5,3,0,
1,Danny,Abbadi,The Assassin,"5' 11""",155 lbs.,--,Orthodox,4,6,0,
2,Nariman,Abbasov,Bayraktar,"5' 8""",155 lbs.,"66.0""",Orthodox,28,4,0,
3,David,Abbott,Tank,"6' 0""",265 lbs.,--,Switch,10,15,0,
4,Hamdy,Abdelwahab,The Hammer,"6' 2""",264 lbs.,"72.0""",Southpaw,5,0,0,
...,...,...,...,...,...,...,...,...,...,...,...
208,Abu,Azaitar,Captain Morocco,"5' 9""",185 lbs.,"76.0""",Orthodox,14,3,1,
209,Ottman,Azaitar,Bulldozer,"5' 8""",155 lbs.,"71.0""",Switch,13,2,0,
210,Luiz,Azeredo,,"5' 9""",154 lbs.,--,Orthodox,15,10,0,
211,Luciano,Azevedo,,"6' 3""",161 lbs.,--,Orthodox,17,9,1,


Dessa forma, a função devolve um dataframe com os lutadores cujos nomes iniciam com a letra definida.

Para alcançar o propósito de obter um dataframe contendo todos os lutadores, faremos um loop através do alfabeto e uma subsequente concatenação das tabelas.

In [4]:
def get_fighters(bruto=False):
    """
    Função que retorna todos os lutadores com dados presentes no site UFCstats.
    """
    # Lista com todas as letras do alfabeto para iteração
    alfabeto = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']

    # Criando df_bruto para armazenar os dados da iteração 
    df_bruto = pd.DataFrame(columns=["First", "Last", "Nickname", "Ht.", "Wt.", "Reach", "Stance", "W", "L", "D", "Belt"])

    for letra in alfabeto:
        # Data frame provisorio que armazena os lutadores da letra da iteração
        df_provisorio = letter_get_fighters(letra)

        # Concatenando o resultado no df_bruto
        df_bruto = pd.concat([df_bruto, df_provisorio], ignore_index=True)

    return df_bruto

In [5]:
# Visualizano aplicação das funções
df = get_fighters()

df

,First,Last,Nickname,Ht.,Wt.,Reach,Stance,W,L,D,Belt
0,Tom,Aaron,,--,155 lbs.,--,,5,3,0,
1,Danny,Abbadi,The Assassin,"5' 11""",155 lbs.,--,Orthodox,4,6,0,
2,Nariman,Abbasov,Bayraktar,"5' 8""",155 lbs.,"66.0""",Orthodox,28,4,0,
3,David,Abbott,Tank,"6' 0""",265 lbs.,--,Switch,10,15,0,
4,Hamdy,Abdelwahab,The Hammer,"6' 2""",264 lbs.,"72.0""",Southpaw,5,0,0,
...,...,...,...,...,...,...,...,...,...,...,...
4097,Dave,Zitanick,,--,170 lbs.,--,,5,7,0,
4098,Alex,Zuniga,,--,145 lbs.,--,,6,3,0,
4099,George,Zuniga,,"5' 9""",185 lbs.,--,,3,1,0,
4100,Allan,Zuniga,Tigre,"5' 7""",155 lbs.,"70.0""",Orthodox,13,1,0,


Assim, conseguimos um dataframe completo, mas algumas colunas ainda precisam de ajustes.

## Tratamentos

Ao final dos tratamentos, será possivel obter um dataframe no sistema métrico internacional ou no sistema métrico inglês. Primeiramente será realizado para o sistema métrico internacional.

O primeiro tratamento será transformar a coluna de altura para centimetros:

In [6]:
# Transformando todos os -- da coluna Ht. em np.nan
df['Ht.'] = df["Ht."].apply(lambda x: np.nan if x == "--" else x)


# Criando função que trata e retorna valor em centimetros
def feet_cm(x):
    if x == np.nan:
        return np.nan
    else:
        feet = float(str(x).split(" ")[0].replace("'", ""))
        inches = float(str(x).split(" ")[-1].replace('"', ''))

        cm = (feet*30.48) + (inches*2.54)

        return cm

# Criando coluna height com a transformação correta
df["height"] = df['Ht.'].apply(feet_cm)
df

,First,Last,Nickname,Ht.,Wt.,Reach,Stance,W,L,D,Belt,height
0,Tom,Aaron,,NaN,155 lbs.,--,,5,3,0,,NaN
1,Danny,Abbadi,The Assassin,"5' 11""",155 lbs.,--,Orthodox,4,6,0,,180.34
2,Nariman,Abbasov,Bayraktar,"5' 8""",155 lbs.,"66.0""",Orthodox,28,4,0,,172.72
3,David,Abbott,Tank,"6' 0""",265 lbs.,--,Switch,10,15,0,,182.88
4,Hamdy,Abdelwahab,The Hammer,"6' 2""",264 lbs.,"72.0""",Southpaw,5,0,0,,187.96
...,...,...,...,...,...,...,...,...,...,...,...,...
4097,Dave,Zitanick,,NaN,170 lbs.,--,,5,7,0,,NaN
4098,Alex,Zuniga,,NaN,145 lbs.,--,,6,3,0,,NaN
4099,George,Zuniga,,"5' 9""",185 lbs.,--,,3,1,0,,175.26
4100,Allan,Zuniga,Tigre,"5' 7""",155 lbs.,"70.0""",Orthodox,13,1,0,,170.18


Realizando tratamento semelhante para a coluna de alcance(Reach).

In [7]:
# Transformando todos os -- da coluna Reach. em np.nan
df['Reach'] = df["Reach"].apply(lambda x: np.nan if x == "--" else x)

# Criando coluna de alcance em cm
df['reach'] = df['Reach'].apply(lambda x: float(str(x).replace('"', ""))*2.54 if x != np.nan else np.nan)

df

,First,Last,Nickname,Ht.,Wt.,Reach,Stance,W,L,D,Belt,height,reach
0,Tom,Aaron,,NaN,155 lbs.,NaN,,5,3,0,,NaN,NaN
1,Danny,Abbadi,The Assassin,"5' 11""",155 lbs.,NaN,Orthodox,4,6,0,,180.34,NaN
2,Nariman,Abbasov,Bayraktar,"5' 8""",155 lbs.,"66.0""",Orthodox,28,4,0,,172.72,167.64
3,David,Abbott,Tank,"6' 0""",265 lbs.,NaN,Switch,10,15,0,,182.88,NaN
4,Hamdy,Abdelwahab,The Hammer,"6' 2""",264 lbs.,"72.0""",Southpaw,5,0,0,,187.96,182.88
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4097,Dave,Zitanick,,NaN,170 lbs.,NaN,,5,7,0,,NaN,NaN
4098,Alex,Zuniga,,NaN,145 lbs.,NaN,,6,3,0,,NaN,NaN
4099,George,Zuniga,,"5' 9""",185 lbs.,NaN,,3,1,0,,175.26,NaN
4100,Allan,Zuniga,Tigre,"5' 7""",155 lbs.,"70.0""",Orthodox,13,1,0,,170.18,177.80


Agora, transformando a coluna de peso em kg.

In [8]:
# Transformando coluna de peso de Ibs em kg

df['weight'] = df['Wt.'].apply(lambda x: round(float(str(x).split(" ")[0])*0.453592, 3) if x != "--" else np.nan)

df

,First,Last,Nickname,Ht.,Wt.,Reach,Stance,W,L,D,Belt,height,reach,weight
0,Tom,Aaron,,NaN,155 lbs.,NaN,,5,3,0,,NaN,NaN,70.307
1,Danny,Abbadi,The Assassin,"5' 11""",155 lbs.,NaN,Orthodox,4,6,0,,180.34,NaN,70.307
2,Nariman,Abbasov,Bayraktar,"5' 8""",155 lbs.,"66.0""",Orthodox,28,4,0,,172.72,167.64,70.307
3,David,Abbott,Tank,"6' 0""",265 lbs.,NaN,Switch,10,15,0,,182.88,NaN,120.202
4,Hamdy,Abdelwahab,The Hammer,"6' 2""",264 lbs.,"72.0""",Southpaw,5,0,0,,187.96,182.88,119.748
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4097,Dave,Zitanick,,NaN,170 lbs.,NaN,,5,7,0,,NaN,NaN,77.111
4098,Alex,Zuniga,,NaN,145 lbs.,NaN,,6,3,0,,NaN,NaN,65.771
4099,George,Zuniga,,"5' 9""",185 lbs.,NaN,,3,1,0,,175.26,NaN,83.915
4100,Allan,Zuniga,Tigre,"5' 7""",155 lbs.,"70.0""",Orthodox,13,1,0,,170.18,177.80,70.307


Após estes tratamentos, podemos excluir as colunas ["Ht.", "wt.", "Reach"].

In [9]:
# Excluindo colunas
df = df.drop(columns=["Ht.", "Wt.", "Reach"])

df

,First,Last,Nickname,Stance,W,L,D,Belt,height,reach,weight
0,Tom,Aaron,,,5,3,0,,NaN,NaN,70.307
1,Danny,Abbadi,The Assassin,Orthodox,4,6,0,,180.34,NaN,70.307
2,Nariman,Abbasov,Bayraktar,Orthodox,28,4,0,,172.72,167.64,70.307
3,David,Abbott,Tank,Switch,10,15,0,,182.88,NaN,120.202
4,Hamdy,Abdelwahab,The Hammer,Southpaw,5,0,0,,187.96,182.88,119.748
...,...,...,...,...,...,...,...,...,...,...,...
4097,Dave,Zitanick,,,5,7,0,,NaN,NaN,77.111
4098,Alex,Zuniga,,,6,3,0,,NaN,NaN,65.771
4099,George,Zuniga,,,3,1,0,,175.26,NaN,83.915
4100,Allan,Zuniga,Tigre,Orthodox,13,1,0,,170.18,177.80,70.307


Uma métrica crucial a considerar é o número total de lutas, determinado pela soma das vitórias, derrotas e empates. Ao extrair dados do site do UFC, essas colunas foram identificadas como "object" em termos de formato. Por isso, é primordial ajustar esse formato antes de prosseguir.

In [10]:
# Ajustando formato das colunas
df['win'] = df["W"].astype("int64")

df['lose'] = df["L"].astype("int64")

df['draw'] = df["D"].astype("int64")

# Criando coluna de número total de lutas no ufc
df['total_fights'] = df['win'] + df["lose"] + df["draw"]

# Apagando colunas W,L e D
df = df.drop(columns=["W", "L", "D"])

df

,First,Last,Nickname,Stance,Belt,height,reach,weight,win,lose,draw,total_fights
0,Tom,Aaron,,,,NaN,NaN,70.307,5,3,0,8
1,Danny,Abbadi,The Assassin,Orthodox,,180.34,NaN,70.307,4,6,0,10
2,Nariman,Abbasov,Bayraktar,Orthodox,,172.72,167.64,70.307,28,4,0,32
3,David,Abbott,Tank,Switch,,182.88,NaN,120.202,10,15,0,25
4,Hamdy,Abdelwahab,The Hammer,Southpaw,,187.96,182.88,119.748,5,0,0,5
...,...,...,...,...,...,...,...,...,...,...,...,...
4097,Dave,Zitanick,,,,NaN,NaN,77.111,5,7,0,12
4098,Alex,Zuniga,,,,NaN,NaN,65.771,6,3,0,9
4099,George,Zuniga,,,,175.26,NaN,83.915,3,1,0,4
4100,Allan,Zuniga,Tigre,Orthodox,,170.18,177.80,70.307,13,1,0,14


#### Primary Key

A chave primária para essa tabela será determinada de forma distinta. Enquanto na tabela de eventos temos uma sequência cronológica clara, facilitando a geração da chave primária conforme novos eventos são registrados, para os lutadores esse processo é mais intrincado. Para contornar essa questão, será gerado um arquivo CSV inicial com as chaves estabelecidas em 10/09/2023. Conforme novos lutadores forem integrados ao banco de dados, uma comparação com o arquivo original será realizada, e esse arquivo será atualizado para referências subsequentes.

***`OBSERVAÇÃO`***
        O código abaixo só deve ser executado uma vez. Ele será comentado para evitar problemas de subcrição do arquivo original. Deve ser utilizado apenas para entender o funcionamento da criação de primary keys dos lutadores.

`-----------------------------------------------------------------------------------------------------------------------`

A estratégia para a criação desta chave primária combina o número total de lutas e as três primeiras letras da palavra "fighters", resultando no formato:

<center>FIGXXXX</center>
Os dígitos "XXXX" representam a ordem no dataframe, baseando-se no número total de lutas. Assim, o lutador com o maior número de lutas no dia da criação do dataframe de referência será identificado como FIG0001, e assim sucessivamente para os demais.

In [11]:
"""# Criando as primary keys dos lutadores

# Obtendo as colunas de referencia 
df_primary_keys = df[["First", "Last", "Nickname", "total_fights"]]

# Criando coluna de comparação para o left join
df_primary_keys['full_name_conference'] = df['First']+df["Last"]+df["Nickname"]

# Ordenando pelo numero de lutas
df_primary_keys = df_primary_keys.sort_values("total_fights")

# Criando lista de index invertido
inverted_index = df.index[::-1] + 1

# Criando as primary keys
df_primary_keys['fighters_id'] =  ["FIG" + f"{index:04}" for index in inverted_index]

# Escrevendo arquivo csv de referencia

df_primary_keys.to_csv("fighters_primary_key_reference.csv")"""

'# Criando as primary keys dos lutadores\n\n# Obtendo as colunas de referencia \ndf_primary_keys = df[["First", "Last", "Nickname", "total_fights"]]\n\n# Criando coluna de comparação para o left join\ndf_primary_keys[\'full_name_conference\'] = df[\'First\']+df["Last"]+df["Nickname"]\n\n# Ordenando pelo numero de lutas\ndf_primary_keys = df_primary_keys.sort_values("total_fights")\n\n# Criando lista de index invertido\ninverted_index = df.index[::-1] + 1\n\n# Criando as primary keys\ndf_primary_keys[\'fighters_id\'] =  ["FIG" + f"{index:04}" for index in inverted_index]\n\n# Escrevendo arquivo csv de referencia\n\ndf_primary_keys.to_csv("fighters_primary_key_reference.csv")'

Com a criação das primary keys de referencia, podemos utilizar este arquivo para criar as primary keys.

In [12]:
# Testando código para lutadores novos
"""new_fake_fighters = pd.DataFrame({
    "First": ["John", "Maicon"],
    "Last": ["Truster", "John"], 
    "Nickname": ["Bart", "Killer"],
    "Stance" : ["Orthodox", "Orthodox"],
    "Belt": [np.nan, np.nan],
    "height": [172.72, 187.96],
    "reach": [182.88, 182.88],
    "weight": [70.307, 70.307],
    "win": [10, 0],
    "lose": [0, 10],
    "draw": [1, 0],
    "total_fights": [11, 10]}
)

new_fake_fighters


df = pd.concat([df, new_fake_fighters], ignore_index=True)

df"""

'new_fake_fighters = pd.DataFrame({\n    "First": ["John", "Maicon"],\n    "Last": ["Truster", "John"], \n    "Nickname": ["Bart", "Killer"],\n    "Stance" : ["Orthodox", "Orthodox"],\n    "Belt": [np.nan, np.nan],\n    "height": [172.72, 187.96],\n    "reach": [182.88, 182.88],\n    "weight": [70.307, 70.307],\n    "win": [10, 0],\n    "lose": [0, 10],\n    "draw": [1, 0],\n    "total_fights": [11, 10]}\n)\n\nnew_fake_fighters\n\n\ndf = pd.concat([df, new_fake_fighters], ignore_index=True)\n\ndf'

In [13]:
# Conferindo se temos lutadores novos
# Criando coluna parcial
df['full_name_conference'] = (df["First"] + df["Last"] + df["Nickname"])

# Lendo arquivo csv de referencia

df_reference_keys = pd.read_csv("fighters_primary_key_reference.csv")

# Fazendo conferencia de nomes
old_fighters_names = df_reference_keys['full_name_conference'].to_list()
new_fighters_names = df['full_name_conference'].to_list()

no_primary_key_fighters = [name for name in new_fighters_names if name not in old_fighters_names]

if len(no_primary_key_fighters) > 0:
    print(f"Os seguintes lutadores não possuem primarys keys: {no_primary_key_fighters}")

# Adicionando primary_key ao dataframe
df = df.merge(df_reference_keys[["full_name_conference", "fighters_id"]], on="full_name_conference", how="left")

df = df.drop(columns=["full_name_conference"])

df

E assim definimos a estrutura final do dataframe. Se um lutador novo, sem uma chave primária registrada, for adicionado à base de dados, um alerta será gerado.

In [16]:
# Renomeando as colunas para manter padrão de letras minusculas 
df.rename(columns={"First": "first", "Last":"last", "Nickname": "nickname", "Stance": "stance", "Belt": "belt"})

df

,First,Last,Nickname,Stance,Belt,height,reach,weight,win,lose,draw,total_fights,fighters_id
0,Tom,Aaron,,,,NaN,NaN,70.307,5,3,0,8,FIG3286
1,Danny,Abbadi,The Assassin,Orthodox,,180.34,NaN,70.307,4,6,0,10,FIG2942
2,Nariman,Abbasov,Bayraktar,Orthodox,,172.72,167.64,70.307,28,4,0,32,FIG0482
3,David,Abbott,Tank,Switch,,182.88,NaN,120.202,10,15,0,25,FIG0923
4,Hamdy,Abdelwahab,The Hammer,Southpaw,,187.96,182.88,119.748,5,0,0,5,FIG3664
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4099,Dave,Zitanick,,,,NaN,NaN,77.111,5,7,0,12,FIG2609
4100,Alex,Zuniga,,,,NaN,NaN,65.771,6,3,0,9,FIG3121
4101,George,Zuniga,,,,175.26,NaN,83.915,3,1,0,4,FIG3843
4102,Allan,Zuniga,Tigre,Orthodox,,170.18,177.80,70.307,13,1,0,14,FIG2267


Com isso temos o dataframe final gerado pela função get_fighters().

In [ ]:
def get_fighters(bruto=False):
    """
    Função que retorna todos os lutadores com dados presentes no site UFCstats.
    """
    # Lista com todas as letras do alfabeto para iteração
    alfabeto = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']

    # Criando df_bruto para armazenar os dados da iteração 
    df_bruto = pd.DataFrame(columns=["First", "Last", "Nickname", "Ht.", "Wt.", "Reach", "Stance", "W", "L", "D", "Belt"])

    for letra in alfabeto:
        # Data frame provisorio que armazena os lutadores da letra da iteração
        df_provisorio = letter_get_fighters(letra)

        # Concatenando o resultado no df_bruto
        df_bruto = pd.concat([df_bruto, df_provisorio], ignore_index=True)


    # Criando instancia para retornar df_bruto
    if bruto:
        return df_bruto

    """
    Abaixo estão os tratamentos realizados até o retorno do df_final
    """

    # Criando cópia do df_bruto que será retornado ao final da função
    df_final = df_bruto.copy()

    # Primeiro tratamento será a criação da coluna height
    # Transformando todos os -- da coluna Ht. em np.nan
    df_final['Ht.'] = df_final["Ht."].apply(lambda x: np.nan if x == "--" else x)


    # Criando função que trata e retorna valor em centimetros
    def feet_cm(x):
        if x == np.nan:
            return np.nan
        else:
            feet = float(str(x).split(" ")[0].replace("'", ""))
            inches = float(str(x).split(" ")[-1].replace('"', ''))

            cm = (feet*30.48) + (inches*2.54)

            return cm

    # Criando coluna height com a transformação correta
    df_final["height"] = df_final['Ht.'].apply(feet_cm)


    # Realizando tratamento da coluna de alcance (Reach)
    # Transformando todos os -- da coluna Reach. em np.nan
    df_final['Reach'] = df_final["Reach"].apply(lambda x: np.nan if x == "--" else x)

    # Criando coluna de alcance em cm
    df_final['reach'] = df_final['Reach'].apply(lambda x: float(str(x).replace('"', ""))*2.54 if x != np.nan else np.nan)


    # Tratamento na coluna de peso
    df_final['weight'] = df_final['Wt.'].apply(lambda x: round(float(str(x).split(" ")[0])*0.453592, 3) if x != "--" else np.nan)


    # Excluindo colunas desnecessarias
    df_final = df_final.drop(columns=["Ht.", "Wt.", "Reach"])


    # Criando coluna de total fights e tratando formato
    # Ajustando formato das colunas
    df_final['win'] = df_final["W"].astype("int64")

    df_final['lose'] = df_final["L"].astype("int64")

    df_final['draw'] = df_final["D"].astype("int64")

    # Criando coluna de número total de lutas no ufc
    df_final['total_fights'] = df_final['win'] + df_final["lose"] + df_final["draw"]

    # Apagando colunas W,L e D
    df_final = df_final.drop(columns=["W", "L", "D"])



    """
    Com todas as colunas criadas, a parte do código a seguir cria primary keys para esta tabela

    OBS: Para que esta parte funcione o arquivo fighters_primary_key_reference.csv deve estar na mesma pasta
    """

    # Conferindo se temos lutadores novos
    # Criando coluna parcial
    df_final['full_name_conference'] = (df_final["First"] + df_final["Last"] + df_final["Nickname"])

    # Lendo arquivo csv de referencia
    df_reference_keys = pd.read_csv("fighters_primary_key_reference.csv")

    # Fazendo conferencia de nomes
    old_fighters_names = df_reference_keys['full_name_conference'].to_list()
    new_fighters_names = df_final['full_name_conference'].to_list()

    no_primary_key_fighters = [name for name in new_fighters_names if name not in old_fighters_names]

    if len(no_primary_key_fighters) > 0:
        return print(f"Os seguintes lutadores não possuem primarys keys: {no_primary_key_fighters}")

    # Adicionando primary_key ao dataframe
    df_final = df_final.merge(df_reference_keys[["full_name_conference", "fighters_id"]], on="full_name_conference", how="left")

    # Apagando coluna de referencia para primary keys
    df_final = df_final.drop(columns=["full_name_conference"])

    # Renomeando as colunas para manter padrão de letras minusculas 
    df_final.rename(columns={"First": "first", "Last":"last", "Nickname": "nickname", "Stance": "stance", "Belt": "belt"})

    return df_final

## Atualizações

- Proposta de atualização:

Automatizar a criação de chaves para lutadores novos.

### Atualização 1: Criando instância que força dataframe mesmo com lutadores sem chave

Esta atualização força o retorno do dataframe indepente se tem ou não luta